# Play-by-Play Feature Engineering

Engineers **carry distribution and zone-level efficiency** features from raw NFL play-by-play rushing data, then joins them onto the RB game-log to create a training-ready dataset.

### Input
| Source | Description |
|--------|-------------|
| S3 — `play_by_play` (2018–2025) | One row per rushing play: player, yards, zone, Diff (yards vs. expectation) |
| S3 — `base_stats` (2018–2025) | Game-level RB stats used to build the RB roster and join keys |

### Output
`play_by_play_feature_engineering.csv` — one row per RB per game with rolling features across three perspectives:

| Perspective | Features |
|-------------|----------|
| **Player** | Carry-bucket ratios (≤0, 1–2, 3–5, 6+, 10+, 20+, 40+ yds) + zone-level Diff rolling means |
| **Team** | Same carry/zone features for the full offensive unit |
| **Opponent** | Same carry/zone features for the defence faced |

> **Leak prevention:** all rolling windows use `shift(1)` — features for game _N_ reflect only games _N-1, N-2, …_

### Pipeline Overview
1. Load raw PBP data and RB roster from S3
2. Normalize player names; map PBP play details → known RB
3. Extract and canonicalize run zones from play Detail strings
4. Aggregate to player-, team-, and opponent-game level
5. Compute rolling features at 1-, 3-, and 5-game windows
6. Join all three rolling lookups onto `base_stats` rows
7. Export to CSV

In [ ]:
# Standard library imports
import pandas as pd


## 1. Environment Setup & Data Ingestion

Add the repo root to `sys.path`, import shared utilities, then load raw play-by-play and base game-log data from S3.

In [ ]:
# Resolve the repo root dynamically so the shared `utils` module can be imported
# regardless of where the notebook is run from within the project tree.
#
# NOTE: `utils` is a private module NOT included in this repository.
# It contains custom web-scraping functions and an S3 client wrapper used
# to fetch pre-scraped NFL data stored in a private S3 bucket. To run this
# notebook you will need to supply your own data source and adapt
# `utils.rush_yard_stats_from_s3()` accordingly.
import sys
from pathlib import Path
repo_root = Path.cwd().resolve().parents[3]
print(f"Adding {repo_root} to sys.path")
sys.path.append(str(repo_root))
import utils


Adding /home/mrmath/sports_betting_empire/sports_betting_empire to sys.path


In [ ]:
# Load play-by-play rushing data from S3 for seasons 2018-2025.
# Each row represents a single rushing play with fields like Detail, Yds, Diff, zone, etc.
pbp_stats = utils.rush_yard_stats_from_s3("play_by_play", 2018, 2025)


In [ ]:
# Quick sanity check — inspect the raw play-by-play table
pbp_stats


,Date,Tm,Opp,Quarter,Time,Down,ToGo,Location,Score,Detail,Yds,EPB,EPA,Diff,season
0,2018-11-11,Eagles,Cowboys,2,14:48,1,10,PHI 26,0-3,Josh Adams right end for 29 yards (tackle by A...,29,0.67,2.59,1.92,2018
1,2018-11-18,Eagles,Saints,2,8:49,2,4,NOR 28,6-17,"Josh Adams left guard for 28 yards, touchdown",28,3.58,7.00,3.42,2018
2,2018-09-06,Eagles,Falcons,4,3:57,3,2,ATL 35,10-12,Corey Clement right tackle for 21 yards (tackl...,21,2.54,4.65,2.11,2018
3,2018-10-28,Eagles,Jaguars,3,11:30,2,6,PHI 22,10-6,Josh Adams right guard for 21 yards (tackle by...,21,0.14,1.80,1.66,2018
4,2018-09-16,Eagles,Buccaneers,3,15:00,1,10,PHI 25,7-20,Jay Ajayi up the middle for 20 yards (tackle b...,20,0.61,1.93,1.32,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116727,2025-12-28,Raiders,Giants,1,10:29,1,10,RAI 16,0-0,Ashton Jeanty up the middle for -2 yards (tack...,-2,-0.14,-0.89,-0.75,2025
116728,2025-12-21,Raiders,Texans,2,6:09,3,1,RAI 45,7-10,Ashton Jeanty left end for -3 yards (tackle by...,-3,1.29,-0.59,-1.88,2025
116729,2025-11-23,Raiders,Browns,2,10:41,2,8,CLE 24,0-14,Geno Smith right tackle for -5 yards (tackle b...,-5,3.57,2.21,-1.36,2025
116730,2025-11-30,Raiders,Chargers,4,8:21,1,1,SDG 1,7-24,Geno Smith up the middle for -5 yards (tackle ...,-5,6.97,4.95,-2.02,2025


In [ ]:
# Load base game-level rushing stats — used here to extract the RB roster
# by date so play-by-play rows can be mapped to a specific player.
base_stats = utils.rush_yard_stats_from_s3("base_stats", 2018, 2025)


In [ ]:
# Collect the full list of team abbreviations present in base_stats.
# Used downstream to validate team-level joins.
teams = base_stats['Team'].unique()


## 2. Player Name Normalisation

Strip generational suffixes (Jr., III, etc.) from player names so they match consistently across the PBP Detail strings and the `base_stats` Player column.  
Then filter `base_stats` to running backs only — this model is scoped to RB rush-yard prediction.

In [ ]:
# Player name normalisation utility.
#
# PBP play Detail strings (e.g. "Derrick Henry Jr. up the middle for 8 yards")
# and the base_stats Player column use inconsistent name formats. Stripping
# generational suffixes makes name-matching reliable across both sources.

import re

def clean_player_name(name: str) -> str:
    """
    Remove generational suffixes from player names.

    Why this matters:
    - Player name keys must be consistent across datasets
    - Some sources include suffixes (e.g., "Jr.", "III")
    - Others omit them
    - Removing them prevents join mismatches and duplicate identities

    Handles:
    - Jr, Jr.
    - Sr, Sr.
    - II, III, IV, V, VI
    - Case-insensitive
    - Extra whitespace
    """

    if not isinstance(name, str):
        return name

    # Normalize whitespace
    name = name.strip()

    # Regex to remove suffix at end of string
    # \b ensures we only match whole suffix tokens
    suffix_pattern = r"\b(JR|SR|II|III|IV|V|VI)\.?$"

    # Remove suffix (case-insensitive)
    cleaned = re.sub(suffix_pattern, "", name, flags=re.IGNORECASE)

    # Remove any leftover trailing spaces
    return cleaned.strip()


In [ ]:
# Normalise player names in base_stats so they can be matched against the
# play Detail strings in pbp_stats (which often omit suffixes like "Jr.").
base_stats['player_name_clean'] = base_stats['Player'].apply(clean_player_name)


In [ ]:
# Restrict base_stats to running backs — we only want to map PBP plays
# to RBs for this workload prediction model.
base_stats = base_stats[base_stats['Pos.'] == "RB"]


## 3. Player-Name Mapping Utilities

PBP plays have no explicit player ID — the ball-carrier is only identified by the prefix of the `Detail` string (e.g. `"Saquon Barkley up the middle for 6 yards"`).  
These utilities resolve that prefix to a known RB from `base_stats`:

- **`build_team_date_lookup`** — pre-indexes `base_stats` into a `date → set(players)` dict for near-O(1) lookups, avoiding repeated full-DataFrame scans.  
- **`map_name_fast`** — for a given PBP row, searches all dates (excluding the current game to avoid trivial self-matches) for a player whose clean name is a prefix of `Detail`.

In [ ]:
# Player-name mapping utilities for linking PBP plays to known RBs.
#
# Problem: PBP Detail strings begin with the ball-carrier's name
# (e.g. "Saquon Barkley up the middle for 6 yards") but don't have an
# explicit player ID column. We need to match that prefix against the
# known RB roster from base_stats.
#
# Solution:
#   1. build_team_date_lookup  — pre-index base_stats into a dict keyed by
#      Date so we can look up active players without scanning the full df.
#   2. map_name_fast           — for a given PBP row, search all OTHER dates
#      for a player whose name is a prefix of the Detail string.
#      (Skipping the same game date avoids trivial self-matches.)

from collections import defaultdict

def build_team_date_lookup(base_stats):
    """
    Precompute team → date → set(player_name_clean)

    Why:
    - Avoid filtering base_stats inside every map_name call
    - Reduces runtime from O(N^2) style scanning
    - Makes name mapping near O(1) lookup per row
    """

    lookup = defaultdict(set)
    for row in base_stats.itertuples(index=False):
        lookup[row.Date].add(row.player_name_clean)

    return lookup

def map_name_fast(row, team_date_lookup):
    """
    Faster name mapping.

    Logic:
    - Get all players on same team
    - Exclude same game date
    - Check if play Detail starts with player name
    """

    game_date = row['Date']
    detail = row['Detail']

    if game_date not in team_date_lookup:
        return None

    for date, players in team_date_lookup.items():

        # Skip same game
        if date == game_date:
            continue

        for player in players:
            if detail.startswith(player):
                return player

    return None


## 4. PBP Preprocessing

Clean up the raw PBP table before player mapping and zone extraction:
- Parse dates so they align with `base_stats`
- Drop plays with no `Detail` string (unresolvable)
- Map each play to its ball-carrier RB using the lookup utilities above
- Drop unmatched plays (QB scrambles, non-RB rushes, name-format failures)
- Build a composite `player_key` (`name_date`) for per-game grouping

In [ ]:
# Parse the Date column to datetime so it aligns with base_stats for lookups
pbp_stats['Date'] = pd.to_datetime(pbp_stats['Date'])


In [ ]:
# Drop plays with no Detail string — these cannot be matched to a player or zone
pbp_stats = pbp_stats[pbp_stats['Detail'].notna()]


In [ ]:
# Build the date-player index then map each PBP play to its ball carrier.
# This is the most compute-intensive step — the lookup dict avoids O(N²) scanning.
date_player_lookup = build_team_date_lookup(base_stats)
pbp_stats['player'] = pbp_stats.apply(lambda row: map_name_fast(row, date_player_lookup), axis=1)


/tmp/ipykernel_23512/1239248517.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pbp_stats['player'] = pbp_stats.apply(lambda row: map_name_fast(row, date_player_lookup), axis=1)


In [ ]:
# Remove plays that couldn't be matched to a known RB —
# these are likely non-RB rushes (QB scrambles, etc.) or name-format mismatches.
pbp_stats = pbp_stats[pbp_stats['player'].notna()]


In [ ]:
# Create a composite player-game key (player_name + date) used to group
# all plays belonging to a single player's game for aggregation below.
pbp_stats['player_key'] = pbp_stats['player'].astype(str) + '_' + pbp_stats['Date'].astype(str)


In [ ]:
# Final deduplication guard — ensure no rows slip through with null Detail or player
pbp_stats = pbp_stats.dropna(subset=['Detail', 'player'])


## 5. Zone Extraction & Normalisation

Parse the run-zone descriptor out of each PBP `Detail` string.  
The Detail format is: `"<Player Name> <zone words> for <N> yards [...]"` — words between the player name and `"for"` form the zone label (e.g. `"up the middle"`, `"left end"`).

Then filter to the eight canonical zones and collapse `"middle"` → `"up the middle"` so every direction has a single consistent label before aggregation.

In [ ]:
# Extract the run-zone descriptor from each PBP Detail string.
#
# Detail format: "<Player Name> <zone words> for <yards> yards [...]"
# e.g. "Derrick Henry up the middle for 8 yards"
#
# Strategy: split on whitespace and collect words that are NOT part of the
# player name and appear BEFORE the word "for". The remaining words form
# the zone (e.g. "up the middle", "left end").

unique_zones = set()
zones = []

for row in pbp_stats.itertuples(index=False):
    player_name = row.player
    detail = row.Detail

    words = detail.split()  # split ONCE
    zone_words = []

    for word in words:
        if word.lower() == "for":
            break

        # If word is not part of player name, treat as zone descriptor
        if word not in player_name.split(" "):
            zone_words.append(word)

    zone = " ".join(zone_words)

    unique_zones.add(zone)
    zones.append(zone)

# Assign column once (much faster than .at in loop)
pbp_stats["zone"] = zones

unique_zones

{'',
 'aborted snap, recovered by Arden Key at TEN-30 (tackle by Dennis Daley)',
 'aborted snap, recovered by Baker Mayfield at MIA-45 (tackle by Raekwon McMillan)',
 'aborted snap, recovered by Bilal Nichols at KC-8 and returned',
 'aborted snap, recovered by Calvin Ridley at TEN-42 (tackle by Andre Cisco)',
 'aborted snap, recovered by Dean Lowry at GB-26 (tackle by Charles Leno)',
 'aborted snap, recovered by Desmond Ridder at CHI-39',
 'aborted snap, recovered by Frank Clark at DEN-28 and returned',
 'aborted snap, recovered by Jalyx Hunt at WAS-28',
 'aborted snap, recovered by Jared Goff at DET-32',
 'aborted snap, recovered by Jerry Jeudy at DEN-10',
 'aborted snap, recovered by Lavonte David at TB-29',
 'aborted snap, recovered by Quinton Jefferson at NYJ-49 and returned',
 'aborted snap, recovered by Trent Murphy at BUF-48 (tackle by Ballage)',
 'aborted snap, recovered by at BAL-30 and returned',
 'aborted snap, recovered by at CHI-39. Penalty on Kenny Clark: Defensive Holdin

In [ ]:
# Keep only plays whose extracted zone matches one of the eight canonical run
# zones (or empty string for unclassified). This filters out edge cases like
# scrambles, reverses, or malformed Detail strings.
good_zones = ['', 'left end', 'left guard', 'left tackle', 'middle',
              'right end', 'right guard', 'right tackle', 'up the middle']
pbp_stats = pbp_stats[pbp_stats['zone'].isin(good_zones)]


In [ ]:
# Zone normalisation map — collapses "middle" into "up the middle" so
# every run direction is represented by a single canonical label.
# This prevents duplicate feature columns in the aggregations below.
zones = {
    'left end': 'left end',
    'left tackle': 'left tackle',
    'left guard': 'left guard',
    'middle': 'up the middle',
    'up the middle': 'up the middle',
    'right guard': 'right guard',
    'right tackle': 'right tackle',
    'right end': 'right end'
}


## 6. Player-Game Level Aggregation

Aggregate all plays for each `(player, game)` group into a single summary row.

**Carry-bucket counts** capture the shape of a player's distribution (not just total yards):

| Column | Carries with... |
|--------|----------------|
| `rushes_less_than_eq_zero` | ≤ 0 yards (stuffed) |
| `rushes_one_to_two` | 1–2 yards |
| `rushes_three_to_five` | 3–5 yards |
| `rushes_six_plus` | 6+ yards |
| `rushes_ten_plus` / `_twenty_plus` / `_forty_plus` | explosive plays |

**Zone-level Diff sums** (`{zone}_diff`) accumulate net yards above/below expectation for each of the 8 run directions, using the play-level `Diff` field from the source data.

In [ ]:
# Aggregate play-by-play data to the player-game level.
#
# For each (player, game) group we compute:
#   - Rush count buckets: ≤0, 1-2, 3-5, 6+, 10+, 20+, 40+ yards
#     These capture the shape of a player's carry distribution, not just totals.
#   - Zone-level Diff sums: net yards above/below expectation for each of the
#     8 run directions. Diff represents yards vs. what was expected on that play
#     based on down/distance context.
#   - total_rushes, total_diff: overall volume and quality summary
#
# Result: play_by_play_player_level_stat_df — one row per player per game

play_by_play_player_level_stats = []
for k, v in pbp_stats.groupby('player_key'):
    player_name = v.iloc[0]['player']
    rushes_less_than_eq_zero = len(v[v['Yds'] <= 0])
    rushes_one_to_two = len(v[(v['Yds'] > 0) & (v['Yds'] <= 2)])
    rushes_three_to_five = len(v[(v['Yds'] > 2) & (v['Yds'] <= 5)])
    rushes_six_plus = len(v[v['Yds'] > 5])
    rushes_ten_plus = len(v[v['Yds'] > 10])
    rushes_twenty_plus = len(v[v['Yds'] > 20])
    rushes_forty_plus = len(v[v['Yds'] > 40])

    # Accumulate Diff by run zone for this player-game
    zone_diff = {}
    for i in range(len(v)):
        zone = v.iloc[i]['zone']
        if zone in zones:
            zone = zones[zone]
        else:
            continue
        diff = v.iloc[i]['Diff']
        if zone not in zone_diff:
            zone_diff[zone] = diff
        else:
            zone_diff[zone] += diff

    data_row = {
        'player_key': k,
        'player': player_name,
        'rushes_less_than_eq_zero': rushes_less_than_eq_zero,
        'rushes_one_to_two': rushes_one_to_two,
        'rushes_three_to_five': rushes_three_to_five,
        'rushes_six_plus': rushes_six_plus,
        'rushes_ten_plus': rushes_ten_plus,
        'rushes_twenty_plus': rushes_twenty_plus,
        'rushes_forty_plus': rushes_forty_plus,
        'total_rushes': len(v),
        'total_diff': v['Diff'].sum(),
        'Date': v.iloc[0]['Date'],
        'Team': v.iloc[0]['Tm'],
        'Opponent': v.iloc[0]['Opp']
    }
    for zone, diff in zone_diff.items():
        data_row[f'{zone}_diff'] = diff
    play_by_play_player_level_stats.append(data_row)

play_by_play_player_level_stat_df = pd.DataFrame(play_by_play_player_level_stats)


In [ ]:
# Create a composite team-game key (team abbreviation + date) so every play
# from the same team in the same game shares a single group identifier.
# Used to aggregate PBP plays to the team level below.
pbp_stats['team_key'] = pbp_stats['Tm'] + '_' + pbp_stats['Date'].dt.strftime('%Y-%m-%d')

## 7. Team-Game Level Aggregation

Mirrors the player-level aggregation, but groups by **offensive team** rather than individual player.  
Captures the team's collective rush distribution for each game — `team_rushes_*` and `team_{zone}_diff` columns.  
These team-level features provide broader offensive context beyond the individual RB's own carries.

In [ ]:
# Aggregate play-by-play data to the team-game level.
#
# Mirrors the player-level aggregation above but groups by the offensive team
# rather than individual player. Captures the team's collective rush distribution
# (carry count buckets + zone-level Diff sums) for a given game. These team-level
# features are joined to each RB row to reflect the broader offensive context.
#
# Result: play_by_play_team_level_stat_df — one row per team per game
play_by_play_team_level_stats = []
for k, v in pbp_stats.groupby('team_key'):
    team_name = v.iloc[0]['Tm']
    rushes_less_than_eq_zero = len(v[v['Yds'] <= 0])
    rushes_one_to_two = len(v[(v['Yds'] > 0) & (v['Yds'] <= 2)])
    rushes_three_to_five = len(v[(v['Yds'] > 2) & (v['Yds'] <= 5)])
    rushes_six_plus = len(v[v['Yds'] > 5])
    rushes_ten_plus = len(v[v['Yds'] > 10])
    rushes_twenty_plus = len(v[v['Yds'] > 20])
    rushes_forty_plus = len(v[v['Yds'] > 40])

    zone_diff = {}
    for i in range(len(v)):
        zone = v.iloc[i]['zone']
        if zone in zones:
            zone = zones[zone]
        else:
            continue
        diff = v.iloc[i]['Diff']
        if zone not in zone_diff:
            zone_diff[zone] = diff
        else:
            zone_diff[zone] += diff

    data_row = {
        'team_key': k,
        'team': team_name,
        'team_rushes_less_than_eq_zero': rushes_less_than_eq_zero,
        'team_rushes_one_to_two': rushes_one_to_two,
        'team_rushes_three_to_five': rushes_three_to_five,
        'team_rushes_six_plus': rushes_six_plus,
        'team_rushes_ten_plus': rushes_ten_plus,
        'team_rushes_twenty_plus': rushes_twenty_plus,
        'team_rushes_forty_plus': rushes_forty_plus,
        'team_total_diff': v['Diff'].sum(),
        'team_total_rushes': len(v),
        'Date': v.iloc[0]['Date'],
        'Team': v.iloc[0]['Tm'],
        'Opponent': v.iloc[0]['Opp'],
    }
    for zone, diff in zone_diff.items():
        data_row[f'team_{zone}_diff'] = diff
    play_by_play_team_level_stats.append(data_row)

play_by_play_team_level_stat_df = pd.DataFrame(play_by_play_team_level_stats)

## 8. Opponent (Defence) Level Aggregation

Groups the same PBP plays by the **defending team** (`Opp`) rather than the offensive team.  
`opponent_rushes_*` and `opponent_{zone}_diff` columns capture how many yards — and where — each defense allowed carries.  
Positive zone-level Diff here means the defense out-performed expectation (bad for the RB); negative means the defense was exploited (favourable matchup signal).

In [ ]:
# Aggregate play-by-play data from the *defensive* (opponent) perspective.
#
# The opponent_key is keyed by the defending team + game date so we capture
# how many rushing yards (and at which zones) each defense allowed. Zone-level
# Diff sums here indicate how far above/below expectation a defense performed
# against runs in each direction — a useful signal when predicting RB workload
# against a specific opponent.
#
# Result: play_by_play_opponent_level_stat_df — one row per opponent per game
play_by_play_opponent_level_stats = []
pbp_stats['opponent_key'] = pbp_stats['Opp'] + '_' + pbp_stats['Date'].dt.strftime('%Y-%m-%d')
for k, v in pbp_stats.groupby('opponent_key'):
    opponent_name = v.iloc[0]['Opp']
    rushes_less_than_eq_zero = len(v[v['Yds'] <= 0])
    rushes_one_to_two = len(v[(v['Yds'] > 0) & (v['Yds'] <= 2)])
    rushes_three_to_five = len(v[(v['Yds'] > 2) & (v['Yds'] <= 5)])
    rushes_six_plus = len(v[v['Yds'] > 5])
    rushes_ten_plus = len(v[v['Yds'] > 10])
    rushes_twenty_plus = len(v[v['Yds'] > 20])
    rushes_forty_plus = len(v[v['Yds'] > 40])

    zone_diff = {}
    for i in range(len(v)):
        zone = v.iloc[i]['zone']
        if zone in zones:
            zone = zones[zone]
        else:
            continue
        diff = v.iloc[i]['Diff']
        if zone not in zone_diff:
            zone_diff[zone] = diff
        else:
            zone_diff[zone] += diff

    data_row = {
        'opponent_key': k,
        'opponent': opponent_name,
        'opponent_rushes_less_than_eq_zero': rushes_less_than_eq_zero,
        'opponent_rushes_one_to_two': rushes_one_to_two,
        'opponent_rushes_three_to_five': rushes_three_to_five,
        'opponent_rushes_six_plus': rushes_six_plus,
        'opponent_rushes_ten_plus': rushes_ten_plus,
        'opponent_rushes_twenty_plus': rushes_twenty_plus,
        'opponent_rushes_forty_plus': rushes_forty_plus,
        'opponent_total_diff': v['Diff'].sum(),
        'opponent_total_rushes': len(v),
        'Date': v.iloc[0]['Date'],
        'Team': v.iloc[0]['Tm'],
        'Opponent': v.iloc[0]['Opp'],
    }
    for zone, diff in zone_diff.items():
        data_row[f'opponent_{zone}_diff'] = diff
    play_by_play_opponent_level_stats.append(data_row)

play_by_play_opponent_level_stat_df = pd.DataFrame(play_by_play_opponent_level_stats)

In [ ]:
# Add a team_key to the opponent-level df so it can later be joined/cross-referenced
# with team-level stats when building the combined training rows.
play_by_play_opponent_level_stat_df['team_key'] = (
    play_by_play_opponent_level_stat_df['Team']
    + '_'
    + play_by_play_opponent_level_stat_df['Date'].dt.strftime('%Y-%m-%d')
)

In [ ]:
# Re-declare the zone normalisation map here to ensure it remains in scope
# when the rolling feature builder cells below are executed independently or
# after a kernel restart. The dict maps raw zone strings to canonical labels.
zones = {
    'left end': 'left end',
    'left tackle': 'left tackle',
    'left guard': 'left guard',
    'middle': 'up the middle',
    'up the middle': 'up the middle',
    'right guard': 'right guard',
    'right tackle': 'right tackle',
    'right end': 'right end',
}

## 9. Rolling Feature Builder

`build_rolling_features` is a generic utility that converts any of the three game-level aggregation DataFrames into a lookup dict of rolling trend features.

**For each entity (player / team / opponent):**
- Sorts by `Date` to ensure chronological order
- Applies `shift(1)` before every rolling window — guaranteeing no current-game leakage
- **Ratio features** (`ratio_cols`): rolling mean of carry-bucket count ÷ total rushes → workload share trend
- **Diff features** (`diff_cols`): rolling mean of zone-level Diff → directional efficiency trend
- Windows: 1-, 3-, and 5-game (controlled by the `windows` parameter)

In [26]:
def build_rolling_features(
    df,
    group_col,
    ratio_cols,
    diff_cols,
    total_col,
    windows=(1, 3, 5)
):
    """
    Generic rolling feature builder.

    - Applies per-group rolling windows
    - Prevents leakage with shift(1)
    - Builds ratio-to-total features
    - Builds directional diff rolling means
    """

    lookup = {}

    for key, g in df.groupby(group_col):

        g = g.sort_values("Date").copy()

        # Precompute total rolling means once per window
        total_roll = {
            w: g[total_col].rolling(w).mean().shift(1)
            for w in windows
        }

        # Ratio features
        for col in ratio_cols:
            for w in windows:
                numerator = g[col].rolling(w).mean().shift(1)
                g[f"{col}_{w}ma"] = numerator / total_roll[w]

        # Diff features
        for col in diff_cols:
            for w in windows:
                g[f"{col}_{w}ma"] = (
                    g[col]
                    .rolling(w)
                    .mean()
                    .shift(1)
                )

        lookup[key] = g

    return lookup


In [ ]:
# Quick sanity check — inspect the opponent-level aggregated stats table
play_by_play_opponent_level_stat_df

,opponent_key,opponent,opponent_rushes_less_than_eq_zero,opponent_rushes_one_to_two,opponent_rushes_three_to_five,opponent_rushes_six_plus,opponent_rushes_ten_plus,opponent_rushes_twenty_plus,opponent_rushes_forty_plus,opponent_total_diff,...,Team,Opponent,opponent_up the middle_diff,opponent_right guard_diff,opponent_left end_diff,opponent_right tackle_diff,opponent_left tackle_diff,opponent_left guard_diff,opponent_right end_diff,team_key
0,49ers_2018-09-09,49ers,4,12,6,5,1,0,0,-9.84,...,Vikings,49ers,-7.42,0.45,-0.74,-0.63,-0.82,-0.68,NaN,Vikings_2018-09-09
1,49ers_2018-09-16,49ers,3,4,7,4,3,1,0,-0.38,...,Lions,49ers,-1.00,0.19,1.66,0.39,-1.48,-0.14,0.00,Lions_2018-09-16
2,49ers_2018-09-23,49ers,3,7,6,3,0,0,0,-0.78,...,Chiefs,49ers,0.79,0.39,NaN,NaN,0.26,-0.03,-2.19,Chiefs_2018-09-23
3,49ers_2018-09-30,49ers,3,7,6,7,5,1,0,0.80,...,Chargers,49ers,-1.54,-0.33,3.30,0.28,-1.25,-0.72,1.06,Chargers_2018-09-30
4,49ers_2018-10-07,49ers,4,5,5,5,0,0,0,-1.08,...,Cardinals,49ers,-3.15,-0.14,NaN,-0.55,1.20,0.69,0.87,Cardinals_2018-10-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249,Washington_2025-12-07,Washington,3,4,14,8,2,0,0,10.38,...,Vikings,Washington,0.34,2.37,2.57,-0.62,0.67,4.49,0.56,Vikings_2025-12-07
4250,Washington_2025-12-14,Washington,2,5,8,5,2,0,0,0.84,...,Giants,Washington,-0.60,-0.27,-0.21,0.65,-1.26,-1.48,0.13,Giants_2025-12-14
4251,Washington_2025-12-20,Washington,5,6,3,11,4,2,1,10.68,...,Eagles,Washington,0.83,3.32,NaN,0.00,7.46,-1.32,0.39,Eagles_2025-12-20
4252,Washington_2025-12-25,Washington,3,10,16,12,2,1,0,3.36,...,Cowboys,Washington,1.55,3.22,-1.78,-0.69,1.21,0.42,-0.57,Cowboys_2025-12-25


## 10. Compute Rolling Features

Call `build_rolling_features` once per perspective to produce three lookup dicts:

| Lookup | Grouped by | Key column |
|--------|-----------|------------|
| `player_pbp_stat_lookup` | individual player | `player_key` |
| `team_pbp_stat_lookup` | offensive team | `team_key` |
| `opponent_pbp_stat_lookup` | defending team | `opponent_key` |

Each dict maps the entity name to its DataFrame with all rolling columns appended.  
The dicts are re-indexed to O(1) row access in the final assembly step.

In [ ]:
# Compute rolling features at three levels of granularity: player, team, and opponent.
#
# For each group we produce 1-, 3-, and 5-game rolling windows (shift(1) prevents leakage):
#   - ratio features: carry-bucket counts divided by total rushes (workload share)
#   - diff features: rolling mean of zone-level Diff (efficiency trend per direction)
#
# Results are stored as dicts keyed by entity name; each value is the group's df
# with the new rolling columns appended. These are indexed in the next cell for
# fast O(1) lookup during the final join loop.
player_pbp_stat_lookup = build_rolling_features(
    play_by_play_player_level_stat_df.fillna(0),
    group_col="player",
    ratio_cols=[
        "rushes_less_than_eq_zero",
        "rushes_one_to_two",
        "rushes_three_to_five",
        "rushes_six_plus",
        "rushes_ten_plus",
        "rushes_twenty_plus",
        "rushes_forty_plus",
    ],
    diff_cols=[
        "left end_diff",
        "left tackle_diff",
        "left guard_diff",
        "up the middle_diff",
        "right guard_diff",
        "right tackle_diff",
        "right end_diff",
        "total_diff",
    ],
    total_col="total_rushes",
)

team_pbp_stat_lookup = build_rolling_features(
    play_by_play_team_level_stat_df.fillna(0),
    group_col="team",
    ratio_cols=[
        "team_rushes_less_than_eq_zero",
        "team_rushes_one_to_two",
        "team_rushes_three_to_five",
        "team_rushes_six_plus",
        "team_rushes_ten_plus",
        "team_rushes_twenty_plus",
        "team_rushes_forty_plus",
    ],
    diff_cols=[
        "team_left end_diff",
        "team_left tackle_diff",
        "team_left guard_diff",
        "team_up the middle_diff",
        "team_right guard_diff",
        "team_right tackle_diff",
        "team_right end_diff",
        "team_total_diff",
    ],
    total_col="team_total_rushes",
)

opponent_pbp_stat_lookup = build_rolling_features(
    play_by_play_opponent_level_stat_df.fillna(0),
    group_col="Opponent",
    ratio_cols=[
        "opponent_rushes_less_than_eq_zero",
        "opponent_rushes_one_to_two",
        "opponent_rushes_three_to_five",
        "opponent_rushes_six_plus",
        "opponent_rushes_ten_plus",
        "opponent_rushes_twenty_plus",
        "opponent_rushes_forty_plus",
    ],
    diff_cols=[
        "opponent_left end_diff",
        "opponent_left tackle_diff",
        "opponent_left guard_diff",
        "opponent_up the middle_diff",
        "opponent_right guard_diff",
        "opponent_right tackle_diff",
        "opponent_right end_diff",
        "opponent_total_diff",
    ],
    total_col="opponent_total_rushes",
)

## 11. Key Alignment & Team Name Mapping

`base_stats` uses short team abbreviations (e.g. `"NE"`) while `pbp_stats` uses full names (e.g. `"New England Patriots"`).  
A mapping is inferred automatically by finding PBP rows that share a `player_key` with `base_stats`, then recording the corresponding full team name — no hardcoded lookup table needed.

The mapped names are then used to build `team_key` and `opponent_key` on `base_stats` so they align with the keys in the rolling-feature lookup dicts.

In [ ]:
# Add a player_key to base_stats using the same (cleaned name + date) format used in
# pbp_stats so the two tables can be matched row-for-row in the join loop below.
base_stats['player_key'] = (
    base_stats['player_name_clean']
    + '_'
    + base_stats['Date'].dt.strftime('%Y-%m-%d')
)

In [ ]:
# Build a mapping from short team abbreviations (used in base_stats, e.g. "NE")
# to the full PBP team names (used in pbp_stats, e.g. "New England Patriots").
#
# Why needed: base_stats uses a different team naming convention than pbp_stats.
# By finding plays where both sources share the same player_key, we can infer
# the correct name for each abbreviation without a hardcoded lookup table.
team_abv_to_name_map = {}
for i in range(len(base_stats)):
    row = base_stats.iloc[i]
    player_key = row.player_key
    curr_pbp_stats = pbp_stats[pbp_stats['player_key'] == player_key]
    if curr_pbp_stats.empty:
        continue
    team_abv_to_name_map[row.Team] = curr_pbp_stats.iloc[0]['Tm']

In [ ]:
# Apply the abbreviation → PBP name mapping to base_stats so Tm and Opponent
# columns use the same team name format as pbp_stats. This allows team_key and
# opponent_key joins to resolve correctly in the final feature assembly loop.
base_stats['Opponent'] = base_stats['Opp'].map(team_abv_to_name_map)
base_stats['Tm'] = base_stats['Team'].map(team_abv_to_name_map)

In [ ]:
# Build team_key and opponent_key on base_stats using the now-aligned PBP team names.
# These keys are used to fetch the pre-computed rolling team and opponent features
# for each RB game row during the final join loop.
base_stats['opponent_key'] = (
    base_stats['Opponent']
    + '_'
    + base_stats['Date'].dt.strftime('%Y-%m-%d')
)
base_stats['team_key'] = (
    base_stats['Tm']
    + '_'
    + base_stats['Date'].dt.strftime('%Y-%m-%d')
)

In [ ]:
# Sort base_stats chronologically (then by team) to ensure the final join loop
# processes rows in game-date order, which is required for rolling window correctness.
base_stats = base_stats.sort_values(["Date", 'Team']).copy()

## 12. Final Feature Assembly

Join the three rolling-feature lookups (player, team, opponent) onto every `base_stats` row to produce the final training DataFrame.

**Performance optimisation — `index_by_key`:** each lookup dict is re-indexed by its key column before the loop, making individual row retrieval O(1) instead of O(N) filter scans.

**For each RB game row:**
1. Look up the player's own rolling PBP stats from `player_indexed`
2. Look up that game's team rolling stats from `team_indexed`
3. Look up the opposing defence's rolling stats from `opponent_indexed`
4. Merge all three into a single flat row (missing entries → empty dict → NaN columns)

Rows with no matching PBP data are logged to stdout so data coverage gaps can be diagnosed.

In [ ]:
# Assemble the final player-game feature rows by joining rolling PBP features
# from all three perspectives (player, team, opponent) onto each base_stats row.
#
# Steps:
#   1. index_by_key — convert each group's df into a dict indexed by its key column
#      so individual row lookups are O(1) rather than O(N) filter scans.
#   2. For each base_stats row, retrieve the pre-computed rolling stats for the
#      player, the player's team, and the opposing defence on that game date.
#   3. Missing entries (no PBP data for that key) are logged and left as empty dicts.
#   4. Merge all three dicts into a single combined row, preserving all key columns.
#
# Result: pbp_train_df — one row per player per game, with full rolling feature set

# Pre-index each lookup by key for O(1) access
def index_by_key(lookup_dict, key_col):
    indexed = {}
    for k, df in lookup_dict.items():
        if key_col in df.columns:
            indexed[k] = df.set_index(key_col)
    return indexed


player_indexed   = index_by_key(player_pbp_stat_lookup,   "player_key")
team_indexed     = index_by_key(team_pbp_stat_lookup,     "team_key")
opponent_indexed = index_by_key(opponent_pbp_stat_lookup, "opponent_key")

rows = []

for row in base_stats.itertuples(index=False):

    player_key   = row.player_key
    team_key     = row.team_key
    opponent_key = row.opponent_key
    player_name  = row.player_name_clean
    team         = row.Tm
    opponent     = row.Opponent

    curr_player = (
        player_indexed.get(player_name, {}).loc[player_key].to_dict()
        if player_name in player_indexed and player_key in player_indexed[player_name].index
        else {}
    )

    curr_team = (
        team_indexed.get(team, {}).loc[team_key].to_dict()
        if team in team_indexed and team_key in team_indexed[team].index
        else {}
    )

    curr_opponent = (
        opponent_indexed.get(opponent, {}).loc[opponent_key].to_dict()
        if opponent in opponent_indexed and opponent_key in opponent_indexed[opponent].index
        else {}
    )

    if not curr_player:
        print(f"Missing data for player_key={player_key}")
    if not curr_team:
        print(f"Missing data for team_key={team_key}")
    if not curr_opponent:
        print(f"Missing data for opponent_key={opponent_key}")

    combined = {
        **curr_player,
        **curr_team,
        **curr_opponent,
        "player_key":   player_key,
        "team_key":     team_key,
        "opponent_key": opponent_key,
    }
    rows.append(combined)

pbp_train_df = pd.DataFrame(rows)

Missing data for player_key=Ito Smith_2018-09-06)
Missing data for player_key=Wendell Smallwood_2018-09-06)
Missing data for player_key=Derrick Coleman_2018-09-09)
Missing data for player_key=Taiwan Jones_2018-09-09)
Missing data for player_key=Michael Burton_2018-09-09)
Missing data for player_key=Benny Cunningham_2018-09-09)
Missing data for player_key=Tra Carson_2018-09-09)
Missing data for player_key=Brandon Wilson_2018-09-09)
Missing data for player_key=Darius Jackson_2018-09-09)
Missing data for player_key=Tyler Ervin_2018-09-09)
Missing data for player_key=Christine Michael_2018-09-09)
Missing data for player_key=Anthony Sherman_2018-09-09)
Missing data for player_key=De'Anthony Thomas_2018-09-09)
Missing data for player_key=Detrez Newsome_2018-09-09)
Missing data for player_key=Brandon Bolden_2018-09-09)
Missing data for player_key=Senorise Perry_2018-09-09)
Missing data for player_key=C.J. Ham_2018-09-09)
Missing data for player_key=Zach Line_2018-09-09)
Missing data for playe

## 13. Finalise & Export

Drop any rows where `player` is null (base_stats entries with no PBP data resolved),  
then write the fully feature-engineered DataFrame to CSV for downstream model training.

In [ ]:
# Drop any remaining rows where the player field is null — these arise when a
# base_stats entry had no matching PBP plays and thus no player name was resolved.
pbp_train_df = pbp_train_df.dropna(subset=['player'])

# Inspect the final training dataframe.
pbp_train_df

,player,rushes_less_than_eq_zero,rushes_one_to_two,rushes_three_to_five,rushes_six_plus,rushes_ten_plus,rushes_twenty_plus,rushes_forty_plus,total_rushes,total_diff,...,opponent_right tackle_diff_3ma,opponent_right tackle_diff_5ma,opponent_right end_diff_1ma,opponent_right end_diff_3ma,opponent_right end_diff_5ma,opponent_total_diff_1ma,opponent_total_diff_3ma,opponent_total_diff_5ma,player_key,opponent_key
0,Devonta Freeman,3.0,0.0,2.0,2.0,1.0,0.0,0.0,7.0,-2.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Devonta Freeman_2018-09-06,Eagles_2018-09-06
1,Tevin Coleman,4.0,3.0,2.0,1.0,0.0,0.0,0.0,10.0,-3.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tevin Coleman_2018-09-06,Eagles_2018-09-06
3,Darren Sproles,2.0,3.0,1.0,0.0,0.0,0.0,0.0,6.0,-1.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Darren Sproles_2018-09-06,Falcons_2018-09-06
4,Jay Ajayi,1.0,5.0,5.0,4.0,1.0,0.0,0.0,15.0,3.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jay Ajayi_2018-09-06,Falcons_2018-09-06
5,Corey Clement,2.0,2.0,1.0,1.0,1.0,1.0,0.0,6.0,1.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Corey Clement_2018-09-06,Falcons_2018-09-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13472,Tony Pollard,1.0,4.0,6.0,3.0,0.0,0.0,0.0,14.0,0.13,...,0.140000,-0.082,0.73,0.376667,0.226,0.91,2.156667,-0.128,Tony Pollard_2026-01-04,Jaguars_2026-01-04
13473,Tyjae Spears,0.0,0.0,1.0,2.0,0.0,0.0,0.0,3.0,1.05,...,0.140000,-0.082,0.73,0.376667,0.226,0.91,2.156667,-0.128,Tyjae Spears_2026-01-04,Jaguars_2026-01-04
13476,Jacory Croskey-Merritt,3.0,5.0,3.0,2.0,0.0,0.0,0.0,13.0,-4.39,...,-0.713333,-0.326,0.00,0.433333,1.050,-3.50,-1.140000,0.250,Jacory Croskey-Merritt_2026-01-04,Eagles_2026-01-04
13477,Chris Rodriguez,1.0,7.0,3.0,5.0,2.0,0.0,0.0,16.0,-0.97,...,-0.713333,-0.326,0.00,0.433333,1.050,-3.50,-1.140000,0.250,Chris Rodriguez_2026-01-04,Eagles_2026-01-04


In [ ]:
# Persist the fully feature-engineered dataset to CSV so it can be loaded by the
# downstream train/test notebooks without re-running this pipeline.
pbp_train_df.to_csv("play_by_play_feature_engineering.csv", index=False)